# 第四章 基于LangChain的文档问答
本章内容主要利用langchain构建向量数据库，可以在文档上方或关于文档回答问题，因此，给定从PDF文件、网页或某些公司的内部文档收集中提取的文本，使用llm回答有关这些文档内容的问题

## 环境配置

安装langchain，设置chatGPT的OPENAI_API_KEY
* 安装langchain
```
pip install langchain
```
* 安装docarray
```
pip install docarray
```
* 设置API-KEY环境变量
```
export OPENAI_API_KEY='api-key'

```

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) #读取环境变量

In [52]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003",max_tokens=1024)
llm("怎么评价人工智能")

'\n\n人工智能是一项极具前景的技术，它的发展正在改变人类的生活方式，带来了无数的便利，也被认为是未来发展的重要标志。人工智能的发展让许多复杂的任务变得更加容易，更高效的完成，节省了大量的时间和精力，为人类发展带来了极大的帮助。'

### 导入embedding模型和向量存储组件
使用Dock Array内存搜索向量存储，作为一个内存向量存储，不需要连接外部数据库

In [3]:
from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
from langchain.chat_models import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.vectorstores import DocArrayInMemorySearch #向量存储
from IPython.display import display, Markdown #在jupyter显示信息的工具

In [4]:
#读取文件
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [24]:
#查看数据
import pandas as pd
data = pd.read_csv(file,header=None)
data

,0,1,2
0,NaN,name,description
1,0.0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
2,1.0,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
3,2.0,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
4,3.0,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
...,...,...,...
996,995.0,"Men's Classic Denim, Standard Fit",Crafted from premium denim that will last wash...
997,996.0,CozyPrint Sweater Fleece Pullover,The ultimate sweater fleece - made from superi...
998,997.0,Women's NRS Endurance Spray Paddling Pants,These comfortable and affordable splash paddli...
999,998.0,Women's Stop Flies Hoodie,This great-looking hoodie uses No Fly Zone Tec...


提供了一个户外服装的CSV文件，我们将使用它与语言模型结合使用

#### 创建向量存储
将导入一个索引，即向量存储索引创建器

In [25]:
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器

In [ ]:
'''
将指定向量存储类,创建完成后，我们将从加载器中调用,通过文档记载器列表加载
'''

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [9]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [21]:
response = index.query(query)#使用索引查询创建一个响应，并传入这个查询

In [23]:
display(Markdown(response))#查看查询返回的内容



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, machine wash and dry, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, handwash, line dry, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

All four shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant

得到了一个Markdown表格，其中包含所有带有防晒衣的衬衫的名称和描述，还得到了一个语言模型提供的不错的小总结

#### 使用语言模型与文档结合使用
想要使用语言模型并将其与我们的许多文档结合使用，但是语言模型一次只能检查几千个单词，如果我们有非常大的文档，如何让语言模型回答关于其中所有内容的问题呢？通过embedding和向量存储实现
* embedding   
文本片段创建数值表示文本语义，相似内容的文本片段将具有相似的向量，这使我们可以在向量空间中比较文本片段
* 向量数据库    
向量数据库是存储我们在上一步中创建的这些向量表示的一种方式，我们创建这个向量数据库的方式是用来自传入文档的文本块填充它。
当我们获得一个大的传入文档时，我们首先将其分成较小的块，因为我们可能无法将整个文档传递给语言模型，因此采用分块embedding的方式储存到向量数据库中。这就是创建索引的过程。

通过运行时使用索引来查找与传入查询最相关的文本片段，然后我们将其与向量数据库中的所有向量进行比较，并选择最相似的n个，返回语言模型得到最终答案

In [26]:
#创建一个文档加载器，通过csv格式加载
loader = CSVLoader(file_path=file)
docs = loader.load()

In [27]:
docs[0]#查看单个文档，我们可以看到每个文档对应于CSV中的一个块

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [31]:
'''
因为这些文档已经非常小了，所以我们实际上不需要在这里进行任何分块,可以直接进行embedding
'''

from langchain.embeddings import OpenAIEmbeddings #要创建可以直接进行embedding，我们将使用OpenAI的可以直接进行embedding类
embeddings = OpenAIEmbeddings() #初始化

In [32]:
embed = embeddings.embed_query("Hi my name is Harrison")#让我们使用embedding上的查询方法为特定文本创建embedding

In [33]:
print(len(embed))#查看这个embedding，我们可以看到有超过一千个不同的元素

1536


In [34]:
print(embed[:5])#每个元素都是不同的数字值，组合起来，这就创建了这段文本的总体数值表示

[-0.021933607757091522, 0.006697045173496008, -0.01819835603237152, -0.039113257080316544, -0.014060650952160358]


In [35]:
'''
为刚才的文本创建embedding，准备将它们存储在向量存储中，使用向量存储上的from documents方法来实现。
该方法接受文档列表、嵌入对象，然后我们将创建一个总体向量存储
'''
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [36]:
query = "Please suggest a shirt with sunblocking"

In [37]:
docs = db.similarity_search(query)#使用这个向量存储来查找与传入查询类似的文本，如果我们在向量存储中使用相似性搜索方法并传入一个查询，我们将得到一个文档列表

In [38]:
len(docs)# 我们可以看到它返回了四个文档

4

In [39]:
docs[0] #，如果我们看第一个文档，我们可以看到它确实是一件关于防晒的衬衫

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

### 如何回答我们文档的相关问题
首先，我们需要从这个向量存储中创建一个检索器，检索器是一个通用接口，可以由任何接受查询并返回文档的方法支持。接下来，因为我们想要进行文本生成并返回自然语言响应


In [40]:
retriever = db.as_retriever() #创建检索器通用接口

In [55]:
llm = ChatOpenAI(temperature = 0.0,max_tokens=1024) #导入语言模型


In [43]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])  # 将合并文档中的所有页面内容到一个变量中


In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") #列出所有具有防晒功能的衬衫并在Markdown表格中总结每个衬衫的语言模型


In [28]:
display(Markdown(response))

| Name | Description |
| --- | --- |
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ sun protection, moisture-wicking, and abrasion-resistant fabric. Recommended by The Skin Cancer Foundation. |
| Men's Plaid Tropic Shirt | Ultracomfortable shirt with UPF 50+ sun protection, wrinkle-free fabric, and front/back cape venting. Made with 52% polyester and 48% nylon. |
| Men's TropicVibe Shirt | Men's sun-protection shirt with built-in UPF 50+ and front/back cape venting. Made with 71% nylon and 29% polyester. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with UPF 50+ sun protection, front/back cape venting, and two front bellows pockets. Made with 100% polyester and is wrinkle-resistant. |

All of these shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. They are made with high-performance fabrics that are moisture-wicking, wrinkle-resistant, and abrasion-resistant. The Men's Plaid Tropic Shirt and Men's Tropical Plaid Short-Sleeve Shirt both have front/back cape venting for added breathability. The Sun Shield Shirt is recommended by The Skin Cancer Foundation as an effective UV protectant.

在此处打印响应，我们可以看到我们得到了一个表格，正如我们所要求的那样

In [56]:
''' 
通过LangChain链封装起来
创建一个检索QA链，对检索到的文档进行问题回答，要创建这样的链，我们将传入几个不同的东西
1、语言模型，在最后进行文本生成
2、传入链类型，这里使用stuff，将所有文档塞入上下文并对语言模型进行一次调用
3、传入一个检索器
'''


qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [46]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."#创建一个查询并在此查询上运行链

In [ ]:
response = qa_stuff.run(query)

In [58]:
display(Markdown(response))#使用 display 和 markdown 显示它



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, machine wash and dry, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, handwash, line dry, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

All four shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant

这两个方式返回相同的结果

#### 不同类型的chain链
想在许多不同类型的块上执行相同类型的问答，该怎么办？之前的实验中只返回了4个文档，如果有多个文档，那么我们可以使用几种不同的方法
* Map Reduce   
将所有块与问题一起传递给语言模型，获取回复，使用另一个语言模型调用将所有单独的回复总结成最终答案，它可以在任意数量的文档上运行。可以并行处理单个问题，同时也需要更多的调用。它将所有文档视为独立的
* Refine    
用于循环许多文档，际上是迭代的，建立在先前文档的答案之上，非常适合前后因果信息并随时间逐步构建答案，依赖于先前调用的结果。它通常需要更长的时间，并且基本上需要与Map Reduce一样多的调用
* Map Re-rank   
对每个文档进行单个语言模型调用，要求它返回一个分数，选择最高分，这依赖于语言模型知道分数应该是什么，需要告诉它，如果它与文档相关，则应该是高分，并在那里精细调整说明，可以批量处理它们相对较快，但是更加昂贵
* Stuff    
将所有内容组合成一个文档